In [2]:
# 필요한 라이브러리 임포트
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from PIL import Image
import os

In [3]:
# 데이터셋에 적용할 변환 정의
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # 이미지 크기 조정
    transforms.ToTensor(),          # 이미지를 텐서로 변환
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # 정규화
])

In [4]:
# ImageFolder를 사용하여 데이터셋 로드
train_dataset = ImageFolder(root='../dataset', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)

val_dataset = ImageFolder(root='../val_dataset', transform=transform)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)

In [5]:
# ResNet50 모델을 불러오고 마지막 레이어를 추가
class CustomResNet(nn.Module):
    def __init__(self, num_classes):
        super(CustomResNet, self).__init__()
        self.resnet = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)
        # 기존 fc 레이어의 출력 크기를 확인하여 새로운 레이어 추가
        num_ftrs = self.resnet.fc.in_features
        # 원래 ResNet50의 마지막 fc 레이어를 제거하고 새로운 레이어 추가
        self.resnet.fc = nn.Sequential(
            nn.Linear(num_ftrs, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        return self.resnet(x)

In [6]:
# train_dataset의 클래스 수를 사용하여 모델 정의
num_classes = 7  # 6개의 동아리 분야로 분류
model = CustomResNet(num_classes=num_classes)

for param in model.resnet.parameters():
    param.requires_grad = False

# 마지막 FC 레이어만 학습하도록 설정
for param in model.resnet.fc.parameters():
    param.requires_grad = True


In [7]:
# 손실 함수 및 옵티마이저 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


CustomResNet(
  (resnet): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
      

In [8]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    
    for epoch in range(num_epochs):
        model.train()  # 모델을 학습 모드로 설정
        running_loss = 0.0

        # 학습 데이터셋으로 모델 학습
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()  # 옵티마이저 초기화
            outputs = model(inputs)  # 모델 예측 값 계산
            loss = criterion(outputs, labels)  # 손실 계산
            loss.backward()  # 역전파
            optimizer.step()  # 가중치 업데이트

            running_loss += loss.item() * inputs.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}')

        # 검증 데이터셋에 대한 평가
        model.eval()  # 모델을 평가 모드로 설정
        val_loss = 0.0
        correct = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs, 1)
                correct += (predicted == labels).sum().item()

        val_loss /= len(val_loader.dataset)
        accuracy = correct / len(val_loader.dataset)
        print(f'Validation Loss: {val_loss:.4f}, Accuracy: {accuracy:.4f}')

In [9]:
#학습
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10)

Epoch 1/10, Loss: 2.1634
Validation Loss: 1.9630, Accuracy: 0.1429
Epoch 2/10, Loss: 2.1663
Validation Loss: 1.9518, Accuracy: 0.1429
Epoch 3/10, Loss: 2.0533
Validation Loss: 1.9493, Accuracy: 0.1429
Epoch 4/10, Loss: 2.2536
Validation Loss: 1.9578, Accuracy: 0.1429
Epoch 5/10, Loss: 2.0115
Validation Loss: 1.9650, Accuracy: 0.1429
Epoch 6/10, Loss: 2.0671
Validation Loss: 1.9658, Accuracy: 0.1429
Epoch 7/10, Loss: 2.0499
Validation Loss: 1.9521, Accuracy: 0.1429
Epoch 8/10, Loss: 2.1214
Validation Loss: 1.9488, Accuracy: 0.1429
Epoch 9/10, Loss: 1.9847
Validation Loss: 1.9482, Accuracy: 0.1429
Epoch 10/10, Loss: 2.0780
Validation Loss: 1.9473, Accuracy: 0.1429


In [10]:
# 학습된 모델 저장
torch.save(model.state_dict(), 'model.pth')